In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import joblib
from sklearn.metrics import r2_score, f1_score, roc_auc_score, precision_score, recall_score, roc_curve
import json

In [ ]:
#In this notebook we create the results required to input in the ensemble model for the long term prediction.

In [22]:
finalx_train_df = pd.read_pickle('./X_train.pickle')
finalx_test_df = pd.read_pickle('./X_test.pickle')
finaly_train_df = pd.read_pickle('./y_train.pickle')
finaly_test_df = pd.read_pickle('./y_test.pickle')
del finalx_train_df['asin']
del finalx_test_df['asin']
del finaly_train_df['asin']
del finaly_test_df['asin']

In [23]:
#Create numpy array out of them
y_train=finaly_train_df.to_numpy()
y_train=y_train[:,0]
y_test=finaly_test_df.to_numpy()
y_test=y_test[:,0]
X_train=finalx_train_df.to_numpy()
X_test=finalx_test_df.to_numpy()

In [24]:
forest = joblib.load('./RandomForestBestModel_FullFeatures_Val.sav')
xgboost = joblib.load('./XgboostBestModel_FullFeatures_Val.sav')

In [25]:
y_pred_forest_test = forest.predict(X_test)
probas_forest_test = forest.predict_proba(X=X_test)[:,1]
y_pred_forest_train = forest.predict(X_train)
probas_forest_train = forest.predict_proba(X=X_train)[:,1]

In [26]:
y_pred_xgboost_test = xgboost.predict(X_test)
probas_xgboost_test = xgboost.predict_proba(X=X_test)[:,1]
y_pred_xgboost_train = xgboost.predict(X_train)
probas_xgboost_train = xgboost.predict_proba(X=X_train)[:,1]

In [27]:
#Produce train test data-frames with new columns of results
y_train_predict_df = pd.read_pickle('./y_train.pickle')
y_test_predict_df = pd.read_pickle('./y_test.pickle')

In [28]:
y_train_predict_df['y_forest_predict']=y_pred_forest_train
y_train_predict_df['y_forest_predict_probas']=probas_forest_train
y_train_predict_df['y_xgboost_predict']=y_pred_xgboost_train
y_train_predict_df['y_xgboost_predict_probas']=probas_xgboost_train
y_train_predict_df.shape

(2768, 6)

In [29]:
y_test_predict_df['y_forest_predict']=y_pred_forest_test
y_test_predict_df['y_forest_predict_probas']=probas_forest_test
y_test_predict_df['y_xgboost_predict']=y_pred_xgboost_test
y_test_predict_df['y_xgboost_predict_probas']=probas_xgboost_test
y_test_predict_df.shape

(923, 6)

In [30]:
Total_res_df=pd.concat([y_test_predict_df, y_train_predict_df], axis=0)

In [31]:
Total_res_df.shape

(3691, 6)

In [32]:
Total_res_df.head()

,asin,label_after_1_yr_period_12_mo_min_bsr,y_forest_predict,y_forest_predict_probas,y_xgboost_predict,y_xgboost_predict_probas
1,B0009DVYVC,1,0,0.411587,0,0.411587
4,B000POZG0U,0,0,0.032897,0,0.032897
9,B0013TOBUW,0,0,0.133685,0,0.133685
11,B0019LVMC4,0,0,0.012275,0,0.012250
16,B002SZNS2M,0,0,0.022496,0,0.022496


In [34]:
Total_res_df.to_pickle('./Total_Valid_res.pickle')